In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import MSELoss
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
import matplotlib.pyplot as plt
import os
import json
from pprint import pprint
import numpy as np
from tqdm import tqdm
import neptune
from PIL import Image

## Parameters

In [3]:
PARAMETERS = {
    "dataset_folder_path": "/media/lorenzo/SAM500/datasets/temp_dataset",
    "n_samples": None,
    "batch_size": 64,
    "n_epochs": 512,
    "lr": 0.00004,
    "lr_decay": 0.999,
    "save_folder": "/media/lorenzo/SAM500/models/gallery-detection/",
}



In [6]:
class GalleryDetectorV3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d = nn.Sequential(
            nn.Conv2d(1, 8, [3, 5], padding=(0, 2), padding_mode="circular", stride=(1, 2)),
            nn.ReLU(),
            nn.Conv2d(8, 16, [3, 5], padding=(0, 2), padding_mode="circular", stride=(1, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, [3, 5], padding=(0, 1), padding_mode="circular", stride=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(32, 16, [3, 3], padding=(0, 1), padding_mode="circular", stride=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(16, 8, [3, 3], padding=(0, 1), padding_mode="circular", stride=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(8, 1, [6, 3], padding=(0, 1), padding_mode="circular", stride=(2, 1)),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(254, 360),
            nn.ReLU(),
        )
        self.conv1d = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2, padding_mode="circular"),
            nn.ReLU(),
            nn.Conv1d(32, 32, 3, padding=1, padding_mode="circular"),
            nn.ReLU(),
            nn.Conv1d(32, 16, 3, padding=1, padding_mode="circular"),
            nn.ReLU(),
            nn.Conv1d(16, 1, 3, padding=1, padding_mode="circular"),
            nn.ReLU(),
        )

    @classmethod
    def is_2d(cls):
        return False

    def forward(self, x):
        x = self.conv2d(x)
        # print("Conv2d")
        # print(torch.mean(x))
        x = torch.flatten(x, 1)
        # print("Flatten")
        # print(torch.mean(x))
        x = self.fc(x)
        # print("Fc")
        # print(torch.mean(x))
        x = torch.unsqueeze(x, 1)
        # print("unsqz")
        # print(torch.mean(x))
        x = self.conv1d(x)
        # print("conv1d")
        # print(torch.mean(x))
        x = torch.flatten(x, 1)
        # print("flatten")
        # print(torch.mean(x))
        return x

## Load dataset

In [7]:
class GalleryDetectionDataset(Dataset):
    def __init__(self, index, n_desired_samples=None):
        self.index = index
        self.get_total_datapoints()
        if n_desired_samples is None:
            self.n_desired_samples = self.n_available_samples
        else:
            if self.n_available_samples > n_desired_samples:
                self.n_desired_samples = n_desired_samples
            else:
                self.n_desired_samples = self.n_available_samples
        self.set_n_samples_per_world()
        self.load()

    def get_total_datapoints(self):
        data = self.index["data"]
        self.n_available_samples = 0
        for world_name in data.keys():
            self.n_available_samples += data[world_name]["n_datapoints"]

    def set_n_samples_per_world(self):
        self.n_samples_per_world = {}
        for world_name in self.index["data"].keys():
            n_samples_in_world = self.index["data"][world_name]["n_datapoints"]
            self.n_samples_per_world[world_name] = int(
                np.round(n_samples_in_world * self.n_desired_samples / self.n_available_samples)
            )
        self.final_n_datapoints = sum(
            self.n_samples_per_world[k] for k in self.n_samples_per_world.keys()
        )
        print(self.n_samples_per_world)

    def load(self):
        print("Allocating memory")
        self.images = torch.zeros(
            (self.final_n_datapoints, 1, 16, self.index["info"]["image_width"])
        )
        self.labels = torch.zeros((self.final_n_datapoints, 360))
        global_index = 0
        with tqdm(total=self.final_n_datapoints) as pbar:
            for world_name in self.index["data"].keys():
                folder_name = self.index["data"][world_name]["images_folder"]
                samples_to_load = self.n_samples_per_world[world_name]
                path_to_world_folder = os.path.join(
                    self.index["info"]["path_to_dataset"], folder_name
                )
                assert os.path.exists(path_to_world_folder)
                raw_idxs = np.arange(0, self.index["data"][world_name]["n_datapoints"])
                np.random.shuffle(raw_idxs)
                idxs = raw_idxs[:samples_to_load]
                for idx in idxs:
                    file_name = f"{idx:010d}.npz"
                    path_to_file = os.path.join(path_to_world_folder, file_name)
                    data = np.load(path_to_file)
                    self.images[global_index, 0, :, :] = torch.tensor(data["image"])
                    self.labels[global_index] = torch.tensor(data["label"])
                    global_index += 1
                    pbar.update(1)
        print("Dataset loaded")

    def __len__(self):
        return self.final_n_datapoints

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

## Train network

### Train

In [9]:
print("Starting neptune run")
print("Getting parameters")
dataset_folder_path = PARAMETERS["dataset_folder_path"]
n_samples = PARAMETERS["n_samples"]
batch_size = PARAMETERS["batch_size"]
n_epochs = PARAMETERS["n_epochs"]
lr = PARAMETERS["lr"]
lr_decay = PARAMETERS["lr_decay"]
save_folder = PARAMETERS["save_folder"]
path_to_index_file = os.path.join(dataset_folder_path, "index.json")
assert os.path.exists(path_to_index_file)
with open(path_to_index_file, "r") as f:
    index = json.load(f)
os.makedirs(save_folder, exist_ok=True)

Starting neptune run
Getting parameters


In [10]:
dataset = GalleryDetectionDataset(index, n_desired_samples=n_samples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

{'env_018': 21777, 'env_007': 13324, 'env_010': 21551, 'env_020': 48033, 'env_017': 21721, 'env_002': 19991, 'env_015': 18613, 'env_008': 27916, 'env_003': 23698, 'env_016': 27254, 'env_005': 31794, 'env_013': 18436, 'env_019': 36708, 'env_004': 28563, 'env_011': 17309, 'env_014': 27147, 'env_006': 34637, 'env_009': 21455, 'env_012': 38653, 'env_001': 24794}
Allocating memory


100%|██████████| 523374/523374 [27:35<00:00, 316.08it/s]

Dataset loaded


In [11]:
model = GalleryDetectorV3()
model = model.type(torch.float)
model = model.to("cuda")
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = ExponentialLR(optimizer, lr_decay)
criterion = MSELoss(reduction="mean")

In [12]:
run = neptune.init_run(
    project="lcano/gallery-detection",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiYjcxZGU4OC00ZjVkLTRmMDAtYjBlMi0wYzkzNDQwOGJkNWUifQ==",
    capture_stderr=False,
    capture_stdout=False,
)  # your credentials

/tmp/ipykernel_8503/85116997.py:1: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/lcano/gallery-detection/e/GAL-60


In [15]:
with tqdm(total = n_epochs * len(dataloader)) as pbar:
    for n_epoch in range(n_epochs):
        epoch_avg_loss = 0
        n_batches = 0
        for batch_data in dataloader:
            n_batches += 1
            img, lbl = batch_data
            img = img.to("cuda").type(torch.float)
            lbl = lbl.to("cuda").type(torch.float)
            optimizer.zero_grad()
            pred = model(img)
            loss = criterion(lbl, pred)
            loss.backward()
            optimizer.step()
            epoch_avg_loss += loss.item()
            run["train/loss"].append(loss.item())
            pbar.update(1)
        lr_scheduler.step()
        epoch_avg_loss /= n_batches
        if epoch_avg_loss < 0.002:
            save_file_path = os.path.join(save_folder, f"{model.__class__.__name__}.torch")
            torch.save(model.state_dict(), save_file_path)
        fig = plt.figure()
        axes1 = fig.add_axes([0, 0, 1, 1])
        axes1.imshow(torch.clone(img[0][0]).detach().cpu().numpy())
        axes2 = fig.add_axes([0, 1, 1, 1])
        axes2.plot(torch.clone(pred[0].detach().cpu()).numpy())
        axes2.plot(torch.clone(lbl[0].detach().cpu()).numpy())
        run["predictions"].append(fig)
        plt.close()

100%|██████████| 4187136/4187136 [46:01:05<00:00, 25.27it/s]   


In [ ]:
run.stop()